In [2]:
# import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re

from sklearn.base import BaseEstimator, TransformerMixin
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model




In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [5]:
tf.__version__

'2.10.0'

In [6]:
# read df 
df = pd.read_csv('./train.csv/train.csv')
test = pd.read_csv('./test.csv/test.csv')

In [7]:
# Top 5 rows of the df
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [8]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [9]:
df.drop(columns='id',inplace=True)

In [10]:
import re

def clean_text(text):
    # Remove newline characters
    text = text.replace('\n', ' ')
    
    # Remove unwanted characters or formatting issues using regex
    text = re.sub(r"[^a-zA-Z0-9\s.,!?']", ' ', text)
    
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

# Apply the cleaning function to the text column
df['comment_text'] = df['comment_text'].apply(clean_text)

In [11]:
df.isnull().sum()

comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [12]:
df['None'] = df['toxic'] + df['identity_hate'] + df['severe_toxic'] + df['obscene'] + df['threat'] + df['insult']

In [13]:
tk = Tokenizer()
tk.fit_on_texts(texts=df['comment_text'])

In [14]:
size = len(tk.word_index) + 1  

In [15]:
df['comment_text'] = tk.texts_to_sequences(df['comment_text'])

In [16]:
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,None
0,"[687, 75, 1, 126, 130, 177, 29, 673, 4510, 120...",0,0,0,0,0,0,0
1,"[93510, 52, 2632, 13, 555, 3806, 74, 4556, 270...",0,0,0,0,0,0,0
2,"[411, 435, 74, 134, 14, 249, 2, 71, 314, 78, 5...",0,0,0,0,0,0,0
3,"[57, 7, 229, 97, 54, 330, 1439, 15, 2122, 7, 6...",0,0,0,0,0,0,0
4,"[6, 1669, 19, 29, 3492, 54, 1070, 6, 579, 40, ...",0,0,0,0,0,0,0


In [17]:
# first we should find which max lenght of instance
max_len = df['comment_text'].str.len().max()

In [18]:
X = pad_sequences(df['comment_text'],maxlen=max_len,padding='post')

In [19]:
y = df.iloc[:,1:]

In [20]:
y = y.to_numpy()

In [21]:
# # Modeling

# model = Sequential()
# model.add(Embedding(size,100,input_length=max_len))
# model.add(LSTM(64))
# model.add(Dense(7, activation='sigmoid'))

# model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1403, 100)         19928800  
                                                                 
 lstm (LSTM)                 (None, 64)                42240     
                                                                 
 dense (Dense)               (None, 7)                 455       
                                                                 
Total params: 19,971,495
Trainable params: 19,971,495
Non-trainable params: 0
_________________________________________________________________


In [22]:
 # model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [23]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [4]:
model = load_model('model.h5')

ValueError: Unrecognized keyword arguments: ['batch_shape']

In [22]:
class CustomTransformer(BaseEstimator, TransformerMixin):

    def __init__(self, param1=None, param2=None):
            
            self.param1 = param1
            self.param2 = param2
            
    def fit(self, X, y=None):
        
        
        return self

    def transform(self, X):
    
        df = X.copy()

        df['comment_text'] = df['comment_text'].apply(prep)
        df['num_sentences'] = df['comment_text'].str.split('\n').str.len()
        df['comment_text'] = df['comment_text'].str.replace('\n',' ')
        df['Num_sp_char'] = df['comment_text'].apply(num_special_char)
        df['None'] = df['toxic'] + df['identity_hate'] + df['severe_toxic'] + df['obscene'] + df['threat'] + df['insult']
        df['comment_text'] = df['comment_text'].str.replace(fr'[{special_charecters}]','',regex=True)
        df['num_words'] = df['comment_text'].str.split().str.len()
        
        
        

        return X_transformed